<a href="https://colab.research.google.com/github/jrg94/CSE5522/blob/lab3/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE 5522 - Lab 3
By Jeremy Grifski

In this lab, we'll take a look at Hidden Markov Models (HMMs) for the Eisner Ice Cream Problem. 

## Part 1: Viterbi Algorithm

Implement the Viterbi algorithm for HMMs for Eisner's Ice Cream Problem (predict whether each day is hot or cold based on the number of ice creams eaten).  Remember that the Viterbi algorithm computes the most likely sequence for an input.

Your solution should be able to handle variable length sequences (in the range of 3-5).

[This zip file has observation probabilities, transition probabilities, and test data for evaluation](https://osu.instructure.com/courses/76815/files/18485497/download).  Please read the probabilities and observations from a file, do not hard-code them. (This is so that we can test with different data/probabilities.)

The observation and transition probabilities have rows being the variable of interest, and columns being the conditioning variables.    For example, P(2|H) is in the 3rd row (including header), 3rd column (including row label).  The columns sum to 1.

The test data has one line per sequence.  When a sequence is less than five observations long, the last columns are filled with zeros.

**1.0**: Let's setup the environment for data loading.

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

**1.1**: Now, we'll need to load all the data from the CSV files.

In [0]:
observation_dataframe = pd.read_csv("observationProbs.csv")
test_dataframe = pd.read_csv("testData.csv")
transition_dataframe = pd.read_csv("transitionProbs.csv")

**1.2**: Let's now take a peak at our data.

In [205]:
display(
    observation_dataframe.shape,
    observation_dataframe.head(), 
    test_dataframe.shape, 
    test_dataframe.head(),
    transition_dataframe.shape,
    transition_dataframe.head()
)

(3, 3)

,P(x|...),C,H
0,1,0.6407,0.0002
1,2,0.1481,0.5341
2,3,0.2122,0.4657


(10, 6)

,SeqNumber,Obs1,Obs2,Obs3,Obs4,Obs5
0,1,2,3,3,2,3
1,2,2,3,2,2,0
2,3,3,1,3,3,1
3,4,2,1,1,0,0
4,5,1,1,1,2,3


(3, 4)

,P(x|...),C,H,START
0,C,0.86,0.07,0.5
1,H,0.07,0.86,0.5
2,STOP,0.07,0.07,0.0


**1.3**: Now, let's build up some strings for indexing the data sets.

In [0]:
COND_PROB_LABEL = "P(x|...)"
HOT_LABEL = "H"
COLD_LABEL = "C"
START_LABEL = "START"
SEQUENCE_NUM_LABEL = "SeqNumber"
LENGTH_LABEL = "length"

**1.4**: With our data loaded, we can begin to construct our M and C matrices—assuming the first sequence for now. Here, we will assume 0 is cold and 1 is hot. This assumption matches the input data tables.  

In [207]:
test_dataframe_copy = test_dataframe.copy()

# Gets everything but seqnumber and adds column with sequence length
cols = test_dataframe_copy.loc[:, test_dataframe_copy.columns != SEQUENCE_NUM_LABEL]  
test_dataframe_copy[LENGTH_LABEL] = cols.astype(bool).sum(axis=1)  

# Computes dimensions of m matrix
m_height = test_dataframe_copy.loc[test_dataframe_copy[SEQUENCE_NUM_LABEL] == 1, LENGTH_LABEL].iloc[0]
m_width = observation_dataframe.shape[1] - 1

# Creates m and c matrix
m = np.zeros(shape=(m_height, m_width))
c = np.zeros(shape=(m_height - 1, m_width))

display(m, c)

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

**1.5**: At this point, we can initialize the first row of the m matrix using the following formula: M<sub>1, k</sub> = π<sub>k</sub>B<sub>k,y<sub>1</sub></sub>. Here, π represents the prior probabilities and B is the emission probability.

We can get π from the START column of our transition matrix. Meanwhile, we can get B from a sequence in our test data and our observation data.  

In [208]:
# Compute prior probabilities from transition matrix
p_hot_given_start = transition_dataframe.loc[transition_dataframe[COND_PROB_LABEL] == HOT_LABEL, START_LABEL].iloc[0]
p_cold_given_start = transition_dataframe.loc[transition_dataframe[COND_PROB_LABEL] == COLD_LABEL, START_LABEL].iloc[0]

# Compute observation 1 from test data 
obs1 = test_dataframe_copy.loc[test_dataframe_copy[SEQUENCE_NUM_LABEL] == 1, cols.columns[0]].iloc[0]

# Compute emission probability from observation 1
p_obs_given_hot = observation_dataframe.loc[observation_dataframe[COND_PROB_LABEL] == obs1, HOT_LABEL].iloc[0]
p_obs_given_cold = observation_dataframe.loc[observation_dataframe[COND_PROB_LABEL] == obs1, COLD_LABEL].iloc[0]

# Compute initial probabilities
m11 = p_cold_given_start * p_obs_given_cold
m12 = p_hot_given_start * p_obs_given_hot

display(
    obs1,
    p_hot_given_start,
    p_cold_given_start,
    p_obs_given_hot,
    p_obs_given_cold,
    m11,
    m12
)

2

0.5

0.5

0.5341

0.1481

0.07405

0.26705

**1.6**: Let's find a way to turn this initialization step into a function.

In [0]:
def prior_probability(transition, label):
  return transition.loc[transition[COND_PROB_LABEL] == label, START_LABEL].iloc[0]

In [0]:
def emission_probability(observation, label, obs):
  return observation.loc[observation[COND_PROB_LABEL] == obs, label].iloc[0]

In [0]:
def test_observation(test, cols, sequence: int, obs: int):
  return test.loc[test[SEQUENCE_NUM_LABEL] == sequence, cols.columns[obs]].iloc[0]

In [0]:
def init_matrices(test, observation, transition, sequence):
  test_copy = test.copy()

  # Gets everything but seqnumber and adds column with sequence length
  COLS = test_copy.loc[:, test_copy.columns != SEQUENCE_NUM_LABEL]  
  test_copy[LENGTH_LABEL] = COLS.astype(bool).sum(axis=1)  

  # Computes dimensions of m matrix
  M_HEIGHT = test_copy.loc[test_copy[SEQUENCE_NUM_LABEL] == sequence, LENGTH_LABEL].iloc[0]
  M_WIDTH = observation.shape[1] - 1

  # Creates m and c matrix
  m = np.zeros(shape=(M_HEIGHT, M_WIDTH))
  c = np.zeros(shape=(M_HEIGHT - 1, M_WIDTH))

  # Compute prior probabilities from transition matrix
  P_COLD_GIVEN_START = prior_probability(transition, COLD_LABEL)
  P_HOT_GIVEN_START = prior_probability(transition, HOT_LABEL)

  # Compute observation 1 from test data 
  OBS1 = test_observation(test_copy, COLS, sequence, 0)

  # Compute emission probability from observation 1
  P_OBS1_GIVEN_COLD = emission_probability(observation, COLD_LABEL, OBS1)
  P_OBS1_GIVEN_HOT = emission_probability(observation, HOT_LABEL, OBS1)

  # Compute initial probabilities
  M11 = P_COLD_GIVEN_START * P_OBS1_GIVEN_COLD
  M12 = P_HOT_GIVEN_START * P_OBS1_GIVEN_HOT

  m[0, 0] = M11
  m[0, 1] = M12

  return m, c

In [213]:
m, c = init_matrices(test_dataframe, observation_dataframe, transition_dataframe, 1)
display(m, c)

array([[0.07405, 0.26705],
       [0.     , 0.     ],
       [0.     , 0.     ],
       [0.     , 0.     ],
       [0.     , 0.     ]])

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

**1.7**: With an initialization function, we can begin tackling the next step: writing a step function for each row of m and c.

In [0]:
def transition_probability(transition, label, given_label):
  return transition.loc[transition[COND_PROB_LABEL] == label, given_label].iloc[0]

In [0]:
def compute_state(p_max_list, p_obs_given_state):
  INDEX = np.argmax(p_max_list)
  P_STATE_TODAY = p_obs_given_state * p_max_list[INDEX]
  return INDEX, P_STATE_TODAY

In [0]:
def step(m, c, test, observation, transition, sequence, obs_index):
  COLS = test.loc[:, test.columns != SEQUENCE_NUM_LABEL]  
  OBS = test_observation(test, COLS, sequence, obs_index)

  # Compute emissions probability: B
  P_OBS_GIVEN_COLD = emission_probability(observation, COLD_LABEL, OBS)
  P_OBS_GIVEN_HOT = emission_probability(observation, HOT_LABEL, OBS)

  # Compute transition probability: A
  P_C_TODAY_GIVEN_C_YESTERDAY = transition_probability(transition, COLD_LABEL, COLD_LABEL)
  P_C_TODAY_GIVEN_H_YESTERDAY = transition_probability(transition, COLD_LABEL, HOT_LABEL)
  P_H_TODAY_GIVEN_C_YESTERDAY = transition_probability(transition, HOT_LABEL, COLD_LABEL)
  P_H_TODAY_GIVEN_H_YESTERDAY = transition_probability(transition, HOT_LABEL, HOT_LABEL)

  # Compute probabilities from previous day
  P_COLD_YESTERDAY = m[obs_index - 1, 0]
  P_HOT_YESTERDAY = m[obs_index - 1, 1]

  # Setup the list of cold today related probabilities
  COLD_PROBS = [
      P_C_TODAY_GIVEN_C_YESTERDAY * P_COLD_YESTERDAY,
      P_C_TODAY_GIVEN_H_YESTERDAY * P_HOT_YESTERDAY
  ]

  HOT_PROBS = [
      P_H_TODAY_GIVEN_H_YESTERDAY * P_HOT_YESTERDAY,
      P_H_TODAY_GIVEN_C_YESTERDAY * P_COLD_YESTERDAY
  ]

  # Compute index and probability for c and m
  C_COLD_VAL, M_COLD_VAL = compute_state(COLD_PROBS, P_OBS_GIVEN_COLD)
  C_HOT_VAL, M_HOT_VAL = compute_state(HOT_PROBS, P_OBS_GIVEN_HOT)

  # Populate c and m
  c[obs_index - 1, :] = C_COLD_VAL, C_HOT_VAL
  m[obs_index, :] = M_COLD_VAL, M_HOT_VAL

In [217]:
step(m, c, test_dataframe, observation_dataframe, transition_dataframe, 1, 1)
display(m, c)

array([[0.07405   , 0.26705   ],
       [0.01351353, 0.10695406],
       [0.        , 0.        ],
       [0.        , 0.        ],
       [0.        , 0.        ]])

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

**1.8**: With the step function written, we can continually generate rows for every observation. 

In [218]:
for i in range(1, m.shape[0]):
  step(m, c, test_dataframe, observation_dataframe, transition_dataframe, 1, i)
display(m, c)

array([[0.07405   , 0.26705   ],
       [0.01351353, 0.10695406],
       [0.00246611, 0.04283531],
       [0.00044407, 0.01967537],
       [0.00029226, 0.00788003]])

array([[0., 0.],
       [0., 0.],
       [1., 0.],
       [1., 0.]])

**1.9**: Now, it's just a matter of selecting the most likely option for the last day. For sequence 1, it's hot (1):

In [219]:
x5 = np.argmax(m[-1])
display(x5)

1

**1.10** With the most likely option, let's no go back and generate the sequence. In this case, we ended up with a sequence of cold, cold, hot, cold, hot for observations 2, 3, 3, 2, 3. 

In [220]:
sequence = list()
sequence.insert(0, x5)
last_index = x5
for i in range(c.shape[0] - 1, -1, -1):
  curr_index = int(c[i][last_index])
  sequence.insert(0, curr_index)
  last_index = curr_index
display(sequence)

[0, 0, 1, 0, 1]

**1.11**: One last thing! Let's turn this sequence generation code into a function for reuse. 




In [0]:
def generate_sequence(c, x):
  sequence = list()
  sequence.insert(0, x)
  last_index = x
  for i in range(c.shape[0] - 1, -1, -1):
    curr_index = int(c[i][last_index])
    sequence.insert(0, curr_index)
    last_index = curr_index
  return sequence

In [222]:
generate_sequence(c, x5)

[0, 0, 1, 0, 1]

**1.12**: Now, for fun, let's run the Viterbi algorithm for all ten sequences:

In [223]:
for i in range(1, test_dataframe.shape[0] + 1):
  m, c = init_matrices(test_dataframe, observation_dataframe, transition_dataframe, i)
  for j in range(1, m.shape[0]):
    step(m, c, test_dataframe, observation_dataframe, transition_dataframe, i, j)
  x = np.argmax(m[-1])
  print(f'Sequence {i}: {generate_sequence(c, x)}')

Sequence 1: [0, 0, 1, 0, 1]
Sequence 2: [0, 0, 0, 1]
Sequence 3: [0, 0, 0, 0, 0]
Sequence 4: [0, 0, 0]
Sequence 5: [0, 0, 0, 0, 0]
Sequence 6: [0, 0, 0, 0]
Sequence 7: [0, 0, 1]
Sequence 8: [0, 1, 0, 0]
Sequence 9: [0, 0, 0, 0, 1]
Sequence 10: [0, 0, 0]


## Part 2: Likelihood Sampling



Using the same network, implement likelihood sampling for approximate inference.  For any test sequence, sample complete sequences of the hidden states n times, where n can range from 10 to 100000 samples. The goal is to approximate the likelihood of all possible sequences.

Assuming the Viterbi sequence is "correct", how long (how many samples) does it take the sampler to converge so that you get the highest match between samples and the Viterbi sequence?

How do I sample a sequence?  In essence, pick a length (3, 4, or 5) - pick the same lengths as each test sample.  Then, sample each weather-day (Hot/Cold) according to the distribution given by the transition network.  You will need to sample Day 1 before sampling Day 2, for example.  You will then have a complete sample of sequence length 3/4/5).  The weight of that sequence sample will be the product of the observation probabilities given the sample (why?).  You can then judge by the overall weight which the most likely weather sequence would be.  Does the best string match your Viterbi answer?

Note: Technically, in the original problem there is the probability of sampling STOP given either HOT or COLD.  For this section of the homework, please just remove the STOP probability and renormalize the other two probabilities so that they sum to one.

**2.0**: In order to begin sampling, we'll need to write a sampling function.

In [0]:
import random

def update_sequence(cold_probability: float, sample_sequence: list):
  random_num = random.random()  
  if random_num < cold_probability:
    sample_sequence.append(0)
  else:
    sample_sequence.append(1)

def sample(test, transition, sequence):
  sample_sequence = list()
  COLS = test.loc[:, test.columns != SEQUENCE_NUM_LABEL]  
  for index, obs in enumerate(COLS):
    OBS = test_observation(test, COLS, sequence, index)
    if OBS != 0: # Sequence continues
      if index == 0: # Use prior probabilities for first iteration
        PRIOR_COLD = prior_probability(transition, COLD_LABEL)
        update_sequence(PRIOR_COLD, sample_sequence)
      else: # Otherwise, use transition probabilities
        if sample_sequence[index - 1] == 0: # Yesterday was cold
          P_C_TODAY_GIVEN_COLD_YESTERDAY = transition_probability(transition, COLD_LABEL, COLD_LABEL)
          update_sequence(P_C_TODAY_GIVEN_COLD_YESTERDAY, sample_sequence)
        else:  # Yesterdat was hot
          P_C_TODAY_GIVEN_HOT_YESTERDAY = transition_probability(transition, COLD_LABEL, HOT_LABEL)
          update_sequence(P_C_TODAY_GIVEN_HOT_YESTERDAY, sample_sequence)
  return sample_sequence

**2.1**: Now that we can generate samples, we'll need to be able to compute a weight for each sample. According to the directions, we should be able to do that by computing the product of the observations probabilities for each element in the sequence:

In [0]:
def weight(test, observations, sample_sequence, sequence):
  product = 1
  COLS = test.loc[:, test.columns != SEQUENCE_NUM_LABEL]  
  for index, state in enumerate(sample_sequence):
    obs = test_observation(test, COLS, sequence, int(index))
    label = COLD_LABEL if state == 0 else HOT_LABEL
    probability = emission_probability(observations, label, obs)
    product *= probability
  return product

**2.2**: With these functions, we're able to generate a sample sequence using random numbers and the transition probabilities. As a result, we can run experiments for each of the sequences to see if we can converge on the viterbi algorithm. As a result, we should write an experiment function which will do that for a given sequence.

In [0]:
def experiment(test: pd.DataFrame, observation: pd.DataFrame, transition: pd.DataFrame, sequence: int, n: int):
  best_sample = []
  best_weight = 0
  for i in range(n):
    s = sample(test, transition, sequence)
    w = weight(test, observation, s, sequence)
    if w > best_weight:
      best_sample = s
      best_weight = w
  return best_sample, best_weight

**2.3**: Now, we can run a few experiments to see if we get any matches.

In [227]:
s, w = experiment(test_dataframe, observation_dataframe, transition_dataframe, 1, 10)
display(s, w)

[1, 1, 1, 1, 1]

0.028811367344428896

In [228]:
s, w = experiment(test_dataframe, observation_dataframe, transition_dataframe, 1, 100)
display(s, w)

[1, 1, 1, 1, 1]

0.028811367344428896

In [229]:
s, w = experiment(test_dataframe, observation_dataframe, transition_dataframe, 1, 1000)
display(s, w)

[1, 1, 1, 1, 1]

0.028811367344428896